### **Jogo dos Oito: Implementação de Métodos de Busca**

<p style="font-size: smaller; text-align: right;">João Dias & Rafael Rodrigues</p>

---

### **Objetivo**

Explorar e implementar dois métodos de busca — um cego (*não informado*) e outro informado — para resolver o problema do Jogo dos Oito. Os métodos escolhidos são **Busca em Amplitude (BFS)** e **Algoritmo A Star**. Este notebook visa comparar a eficiência e o desempenho de ambos os algoritmos, focando na solução do problema e na análise dos resultados.

---

### **Estrutura do Notebook**

1. **Introdução**  
   Contextualização sobre o Jogo dos Oito e os métodos de busca empregados.

2. **Regras do Jogo**  
   Definição do estado inicial, operações válidas, e o estado objetivo.

3. **Implementação da Busca em Amplitude (BFS)**  
   Desenvolvimento passo a passo do método cego.

4. **Implementação do Algoritmo A***  
   Utilização de heurísticas para um método informado mais eficiente.

5. **Visualização dos Movimentos**  
   Demonstração das jogadas que levam do estado inicial ao estado objetivo.

6. **Comparação de Desempenho**  
   Tempo de execução, número de estados explorados e eficiência de cada método.

7. **Conclusão**  
   Reflexões sobre as vantagens e desvantagens de cada abordagem.

---

### **Introdução**

O Jogo dos Oito é um quebra-cabeça clássico que consiste em um tabuleiro de 3x3, com peças numeradas de 1 a 8 e uma posição vazia. O objetivo é reorganizar as peças de um estado inicial para um estado final predeterminado, utilizando movimentos válidos como deslizar uma peça para a posição vazia.

Os métodos de busca empregados para resolver este problema têm aplicações práticas em Inteligência Artificial, especialmente em problemas de caminho mínimo e otimização. Enquanto a **Busca em Amplitude (BFS)** explora todas as possibilidades indiscriminadamente, o **Algoritmo A*** utiliza heurísticas para priorizar caminhos promissores, tornando a busca mais eficiente.

Neste notebook, serão desenvolvidos e comparados ambos os métodos, enfatizando a implementação do zero e a análise dos resultados.

---

## **Implementação**




*<h4>Importação das Bibliotecas*

In [ ]:
# Bibliotecas necessárias
import numpy as np
import heapq
from collections import deque
import time
import matplotlib.pyplot as plt


## **Definindo o estado do Tabuleiro**

*<h4>Criamos a classe PuzzleState para representar o estado do tabuleiro, armazenando o tabuleiro atual, o movimento realizado, o estado anterior e a profundidade atual.*

In [ ]:
class PuzzleState:
    def __init__(self, board, parent=None, move=None, depth=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth

    def to_tuple(self):
        return self.board.tobytes()

    def is_goal(self, goal_state):
        return (self.board == goal_state).all()

    def generate_moves(self, visited):
        x, y = next((i, j) for i, row in enumerate(self.board) for j, val in enumerate(row) if val == 0)
        directions = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}
        moves = []

        for move, (dx, dy) in directions.items():
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = self.board.copy()
                new_board[x, y], new_board[new_x, new_y] = new_board[new_x, new_y], new_board[x, y]
                new_tuple = new_board.tobytes()
                if new_tuple not in visited:
                    moves.append(PuzzleState(new_board, self, move, self.depth + 1))
        return moves

    def __lt__(self, other):
        return False  # Só necessário para o uso do heapq

## **Exibição do Tabuleiro**

*<h4>A função de exibição irá atualizar o tabuleiro de maneira visual a cada movimento. Aqui, estamos utilizando o matplotlib para desenhar o tabuleiro no formato de uma grade.*

In [ ]:
def visualize_board(board):
    fig, ax = plt.subplots(figsize=(3, 3))
    ax.matshow(np.zeros_like(board), cmap="Greys")
    for i in range(3):
        for j in range(3):
            value = board[i][j]
            ax.text(j, i, str(value) if value != 0 else "", ha='center', va='center', fontsize=20)
    plt.axis('off')
    plt.show()

## **Exibindo o caminho final**

In [ ]:
def display_solution_path(state):
    path = []
    while state:
        path.append(state.board)
        state = state.parent
    path.reverse()

    print("Solução encontrada:")
    for board in path:
        visualize_board(board)  

## **Busca em Amplitude (Breadth-First Search)**

*<h4>A busca em amplitude expande os nós de forma nível a nível, visitando todos os vizinhos antes de ir mais fundo. É um método cego, ou seja, não utiliza informações adicionais como heurísticas.*

In [ ]:
def breadth_first_search(initial_state, goal_state):
    visited = set()
    queue = deque([initial_state])

    while queue:
        state = queue.popleft()

        # Exibir o tabuleiro atual
        visualize_board(state.board)

        if state.is_goal(goal_state):
            return state

        visited.add(state.to_tuple())

        for move in state.generate_moves(visited):
            queue.append(move)
    return None

## **Busca A Star**

*<h4>A busca A estrela é um algoritmo informado que utiliza uma heurística para calcular a distância estimada até o objetivo, melhorando a eficiência da busca.*

In [ ]:
def a_star_search(initial_state, goal_state):
    visited = set()
    queue = []
    heapq.heappush(queue, (0, initial_state))

    while queue:
        _, state = heapq.heappop(queue)

        # Exibir o tabuleiro atual
        visualize_board(state.board)

        if state.is_goal(goal_state):
            return state

        visited.add(state.to_tuple())

        for move in state.generate_moves(visited):
            if move.to_tuple() not in visited:
                cost = move.depth + heuristic(move.board, goal_state)
                heapq.heappush(queue, (cost, move))
    return None

## **Função Heurística**

In [ ]:
def heuristic(board, goal_state):
    distance = 0
    for i in range(3):
        for j in range(3):
            if board[i][j] != 0:
                goal_pos = np.argwhere(goal_state == board[i][j])[0]
                x, y = goal_pos
                distance += abs(x - i) + abs(y - j)
    return distance

## **Testando os métodos de busca**

In [ ]:
# Testando os Métodos de Busca

# Estado inicial e objetivo
initial_board = np.array([
    [1, 2, 3],
    [4, 0, 6],
    [7, 5, 8]
])

goal_board = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
])

initial_state = PuzzleState(initial_board)
goal_state = goal_board

# Teste com BFS
print("Executando Busca em Largura (BFS)...")
start_bfs = time.time()  # Marca o tempo inicial para BFS
result_bfs = breadth_first_search(initial_state, goal_state)
end_bfs = time.time()  # Marca o tempo final para BFS
if result_bfs:
    display_solution_path(result_bfs)
bfs_duration = end_bfs - start_bfs  # Tempo gasto na busca BFS
print(f"\nTempo de execução da BFS: {bfs_duration:.6f} segundos\n")

# Teste com A*
print("Executando Busca A*...")
start_a_star = time.time()  # Marca o tempo inicial para A*
result_a_star = a_star_search(initial_state, goal_state)
end_a_star = time.time()  # Marca o tempo final para A*
if result_a_star:
    display_solution_path(result_a_star)
a_star_duration = end_a_star - start_a_star  # Tempo gasto na busca A*
print(f"\nTempo de execução da A*: {a_star_duration:.6f} segundos\n")

# Comparando os tempos de execução
if bfs_duration < a_star_duration:
    print("A Busca em Largura (BFS) foi mais rápida.")
elif a_star_duration < bfs_duration:
    print("A Busca A* foi mais rápida.")
else:
    print("Ambas as buscas tiveram o mesmo tempo de execução.")

## **Conclusão**

Neste notebook, comparamos dois algoritmos de busca aplicados ao Jogo dos Oito:

Busca em Amplitude: Um método cego que explora o espaço de estados sem qualquer informação adicional, mais lento para grandes estados.
Busca A*: Um método informado que utiliza uma heurística, mais eficiente em encontrar soluções rápidas.
Ambos os algoritmos foram implementados e testados, com o tempo de execução sendo medido para avaliar a eficiência de cada abordagem.

